In [1]:
# -*- coding: utf-8 -*-
"""historical_data_in_simple_table_for_n_years.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1RJSU82yAjVUV06cGK7IchzBvXZ3Bk1yc
"""

import requests
import pandas as pd
from datetime import datetime, timedelta
# df=pd.read_csv('/content/ind_nifty50list.csv')
df=pd.read_csv('complete.csv')
df

import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch historical data for a given time period
def fetch_historical_data(symbol, start_date, end_date):
    url = f'https://api.upstox.com/v2/historical-candle/{symbol}/1minute/{end_date}/{start_date}'
    headers = {'Accept': 'application/json'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json().get('data', [])
        return pd.DataFrame(data)
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Function to fetch instrument_keys based on a list of names using boolean indexing
def get_instrument_keys(df, names):
    instrument_keys = {}

    for name in names:
        mask = df['tradingsymbol'] == name
        if mask.any():
            instrument_key = df.loc[mask, 'instrument_key'].iloc[0]
            instrument_keys[name.lower()] = instrument_key
        else:
            print(f"No instrument_key found for name: {name}")

    return instrument_keys

# Example usage for multiple names
names_to_search = ['RELIANCE', 'TATASTEEL', 'INFY']
instrument_keys_result = get_instrument_keys(df, names_to_search)

# Specify the number of years to fetch historical data
num_years = 27

# Dictionary to store DataFrames
dfs = {}

# Loop through each instrument key and fetch historical data
for name, instrument_key in instrument_keys_result.items():
    if instrument_key is not None:
        df_symbol = pd.DataFrame()

        for i in range(num_years):
            end_date = (datetime.today() - timedelta(days=365 * i)).strftime('%Y-%m-%d')
            start_date = (datetime.today() - timedelta(days=365 * (i + 1))).strftime('%Y-%m-%d')

            # Fetch historical data for the current time period
            df_current_period = fetch_historical_data(instrument_key, start_date, end_date)

            # Append the data to the df_symbol DataFrame
            if df_current_period is not None:
                df_symbol = pd.concat([df_symbol, df_current_period], ignore_index=True)

        # Convert timestamp column to datetime format
        if 'timestamp' in df_symbol.columns:
            df_symbol['timestamp'] = pd.to_datetime(df_symbol['timestamp'])

        # Store the DataFrame in the dictionary with a name like df_reliance
        dfs[f"df_{name.lower()}"] = df_symbol

# Access individual DataFrames using dfs dictionary
df_reliance = dfs.get('df_reliance', pd.DataFrame())
df_tatasteel = dfs.get('df_tatasteel', pd.DataFrame())
df_infy = dfs.get('df_infy', pd.DataFrame())

# Print individual DataFrames
print("DataFrame for RELIANCE:")
print(df_reliance)

print("\nDataFrame for TATASTEEL:")
print(df_tatasteel)

print("\nDataFrame for INFY:")
print(df_infy)

import pandas as pd

# Function to clean the data for a given DataFrame
def clean_data(df):
    # Create an empty DataFrame with the desired column names
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'open interest']
    df_cleaned_data = pd.DataFrame(columns=columns)

    # Iterate through each row in the original DataFrame
    for index, row in df.iterrows():
        # Extract data from the list
        row_data = row[0] if isinstance(row[0], list) else row[0][0]  # Assuming each row contains a list or list within a list

        # Reshape the row_data if needed
        if isinstance(row_data, list) and len(row_data) == 1:
            row_data = row_data[0]

        # Create a temporary DataFrame
        temp_df = pd.DataFrame([row_data], columns=columns[:len(row_data)])

        # Append the temporary DataFrame to the cleaned DataFrame
        df_cleaned_data = pd.concat([df_cleaned_data, temp_df], ignore_index=True)

    # Convert timestamp column to datetime format
    df_cleaned_data['timestamp'] = pd.to_datetime(df_cleaned_data['timestamp'])

    return df_cleaned_data

# Example usage for each DataFrame
df_reliance_cleaned = clean_data(df_reliance)
df_tatasteel_cleaned = clean_data(df_tatasteel)
df_infy_cleaned = clean_data(df_infy)

# Print cleaned DataFrames
print("Cleaned DataFrame for RELIANCE:")
print(df_reliance_cleaned)

print("\nCleaned DataFrame for TATASTEEL:")
print(df_tatasteel_cleaned)

print("\nCleaned DataFrame for INFY:")
print(df_infy_cleaned)

# # Calculate the difference between 'high' and 'low' and store it in a new column
# df_cleaned_data['ideal_change_%'] = ((df_cleaned_data['high'] - df_cleaned_data['low'])/(df_cleaned_data['high']+df_cleaned_data['low']))*200
# df_cleaned_data['ideal_change_%']

# import matplotlib.pyplot as plt
# import seaborn as sns
# # # Filter columns with more than 10 unique numeric values
# # numeric_cols = df.select_dtypes(include='number')
# # cols_to_plot = numeric_cols.columns[numeric_cols.nunique() > 2]

# # # Plot histograms and PDFs for selected columns
# # for column in cols_to_plot:
# plt.figure(figsize=(8, 4))
# # Plot histogram
# plt.subplot(1, 2, 1)
# sns.histplot(df_cleaned_data['ideal_change_%'], kde=False, bins=50)
# plt.title(f'Histogram for ideal_change_%')

# # Plot PDF
# plt.subplot(1, 2, 2)
# sns.histplot(df_cleaned_data['ideal_change_%'], kde=True, bins=50, color='orange')
# plt.title(f'PDF for column')
# #print(df[column].skew())

# plt.tight_layout()
# plt.show()

DataFrame for RELIANCE:
                                                 candles
0      [2025-03-07T15:29:00+05:30, 1247.4, 1248, 1246...
1      [2025-03-07T15:28:00+05:30, 1247.55, 1247.75, ...
2      [2025-03-07T15:27:00+05:30, 1247.6, 1248, 1247...
3      [2025-03-07T15:26:00+05:30, 1247.85, 1248, 124...
4      [2025-03-07T15:25:00+05:30, 1247.65, 1248, 124...
...                                                  ...
46930  [2024-09-09T09:19:00+05:30, 1459.3, 1460.6, 14...
46931  [2024-09-09T09:18:00+05:30, 1459.8, 1459.8, 14...
46932  [2024-09-09T09:17:00+05:30, 1461.9, 1462.45, 1...
46933  [2024-09-09T09:16:00+05:30, 1462.1, 1463.8, 14...
46934  [2024-09-09T09:15:00+05:30, 1466.5, 1467.4, 14...

[46935 rows x 1 columns]

DataFrame for TATASTEEL:
                                                 candles
0      [2025-03-07T15:29:00+05:30, 151.6, 151.7, 151....
1      [2025-03-07T15:28:00+05:30, 151.51, 151.7, 151...
2      [2025-03-07T15:27:00+05:30, 151.55, 151.6, 151...
3      [2025

C:\Users\runneradmin\AppData\Local\Temp\ipykernel_7184\1123412874.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_data = row[0] if isinstance(row[0], list) else row[0][0]  # Assuming each row contains a list or list within a list
C:\Users\runneradmin\AppData\Local\Temp\ipykernel_7184\1123412874.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cleaned_data = pd.concat([df_cleaned_data, temp_df], ignore_index=True)


C:\Users\runneradmin\AppData\Local\Temp\ipykernel_7184\1123412874.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_data = row[0] if isinstance(row[0], list) else row[0][0]  # Assuming each row contains a list or list within a list
C:\Users\runneradmin\AppData\Local\Temp\ipykernel_7184\1123412874.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cleaned_data = pd.concat([df_cleaned_data, temp_df], ignore_index=True)


C:\Users\runneradmin\AppData\Local\Temp\ipykernel_7184\1123412874.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row_data = row[0] if isinstance(row[0], list) else row[0][0]  # Assuming each row contains a list or list within a list
C:\Users\runneradmin\AppData\Local\Temp\ipykernel_7184\1123412874.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cleaned_data = pd.concat([df_cleaned_data, temp_df], ignore_index=True)


Cleaned DataFrame for RELIANCE:
                      timestamp     open     high      low    close  volume  \
0     2025-03-07 15:29:00+05:30  1247.40     1248  1246.05  1246.40   42186   
1     2025-03-07 15:28:00+05:30  1247.55  1247.75  1245.40  1247.40   82116   
2     2025-03-07 15:27:00+05:30  1247.60     1248  1247.50  1247.55   71080   
3     2025-03-07 15:26:00+05:30  1247.85     1248  1247.40  1247.55   74696   
4     2025-03-07 15:25:00+05:30  1247.65     1248  1247.60  1247.85   51785   
...                         ...      ...      ...      ...      ...     ...   
46930 2024-09-09 09:19:00+05:30  1459.30   1460.6  1458.00  1460.40   46324   
46931 2024-09-09 09:18:00+05:30  1459.80   1459.8  1457.60  1459.30   46684   
46932 2024-09-09 09:17:00+05:30  1461.90  1462.45  1457.50  1459.75   69474   
46933 2024-09-09 09:16:00+05:30  1462.10   1463.8  1461.00  1462.45   61092   
46934 2024-09-09 09:15:00+05:30  1466.50   1467.4  1462.10  1462.10  150948   

      open interest

In [2]:
import pandas as pd
from datetime import datetime

# Assuming you have already defined and cleaned your DataFrames: df_reliance_cleaned, df_tatasteel_cleaned, df_infy_cleaned

# Function to save DataFrame to Excel with a specific file name
def save_to_excel(df, df_name):
    # Remove timezone information from datetime columns
    df_no_timezone = df.copy()
    df_no_timezone['timestamp'] = df_no_timezone['timestamp'].dt.tz_localize(None)

    file_name = f"{df_name}_historical_1minute_data.xlsx"
    df_no_timezone.to_excel(file_name, index=False)
    print(f"DataFrame '{df_name}' successfully saved to Excel file: {file_name}")

# Save each DataFrame to individual Excel files
save_to_excel(df_reliance_cleaned, "df_reliance")
save_to_excel(df_tatasteel_cleaned, "df_tatasteel")
save_to_excel(df_infy_cleaned, "df_infy")


DataFrame 'df_reliance' successfully saved to Excel file: df_reliance_historical_1minute_data.xlsx


DataFrame 'df_tatasteel' successfully saved to Excel file: df_tatasteel_historical_1minute_data.xlsx


DataFrame 'df_infy' successfully saved to Excel file: df_infy_historical_1minute_data.xlsx


In [3]:
df_tatasteel_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46935 entries, 0 to 46934
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype                    
---  ------         --------------  -----                    
 0   timestamp      46935 non-null  datetime64[ns, UTC+05:30]
 1   open           46935 non-null  float64                  
 2   high           46935 non-null  float64                  
 3   low            46935 non-null  float64                  
 4   close          46935 non-null  float64                  
 5   volume         46935 non-null  object                   
 6   open interest  46935 non-null  object                   
dtypes: datetime64[ns, UTC+05:30](1), float64(4), object(2)
memory usage: 2.5+ MB
